In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

from functools import partial
from bayes_opt import BayesianOptimization

import numpy as np
import random
import sys
import os
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Reading dataset

In [3]:
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read().lower().split('\n')
poem_list = []
raw_text = ''
for j in range(len(text) + 1):
    if j == len(text):
        poem_list.append(raw_text)
    elif text[j] == '':
        if raw_text != '':
            poem_list.append(raw_text)
        raw_text = ''
        continue
    elif text[j][-1].isdigit():
        continue
    else:
        subsentence = text[j] + '\n'
        raw_text += subsentence  
print('Number of poems:', len(poem_list))

Number of poems: 154


## Create Sequences

In [4]:
# organize into sequences of characters
def create_sequence(raw_text, length, step):
    sequences = []
    next_chars = []
    for i in range(0, len(raw_text) - length, step):
        # select sequence of tokens
        seq = raw_text[i:i + length]
        # store
        sequences.append(seq)
        next_chars.append(raw_text[i + length])
    return sequences, next_chars

In [5]:
length = 40
step = 1
sequences = []
next_chars = []
for poem in poem_list:
    sub_sequences, sub_next_chars = create_sequence(poem, length, step)
    sequences += sub_sequences
    next_chars += sub_next_chars
print('Total Sequences: %d' % len(sequences))

Total Sequences: 88130


## Character Mappings and Inverse Mappings

In [6]:
poem_string = "".join(poem_list)
chars = sorted(list(set(poem_string)))
char_index_map = dict((c, i) for i, c in enumerate(chars))
index_char_map = dict((i, c) for i, c in enumerate(chars))
vocab_size = len(char_index_map)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 38


## Vectorization

In [7]:
X = np.zeros((len(sequences), length, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, char in enumerate(sentence):
        X[i, t, char_index_map[char]] = 1
    y[i, char_index_map[next_chars[i]]] = 1    

## Build the RNN Training Model

In [8]:
import warnings
warnings.filterwarnings('ignore')

print('Build model...')
model = Sequential()
model.add(Conv1D(filters = 64,
                 kernel_size = 5,
                 padding='valid',
                 activation='relu',
                 strides=1, input_shape=(length, len(chars))))
model.add(MaxPooling1D(pool_size = 2))
model.add(BatchNormalization())
model.add(LSTM(128))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.summary()   

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Build model...





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 36, 64)            12224     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 18, 64)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 18, 64)            256       
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                4902      
_________________________________________________________________
activation_1 (Activation)    (None, 38)                0         
Total params: 116,198
Trainable params: 116,070
Non-trainable params: 128
________________________________________________

## Sampling function (combining softmax with temperature)

In [9]:
def sample(preds, temperature):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Training the model and generating outputs

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for iteration in range(1):
    model.fit(X, y, batch_size=128, epochs=50) ############ 50 epochs

    for temperature in [1.5, 0.75, 0.25]:
        print()
        print('----- temperature parameter:', temperature)

        generated = ''
        sentence = "shall i compare thee to a summer's day?\n"
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        line_count = 0
        while (line_count <= 12):
            x = np.zeros((1, length, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_index_map[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = index_char_map[next_index]
            
            if next_char == '\n':
                line_count += 1

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Epoch 1/50
88130/88130 [==============================] - 51s 573us/step - loss: 1.7970 - acc: 0.4644
Epoch 2/50
88130/88130 [==============================] - 33s 369us/step - loss: 1.6771 - acc: 0.4935
Epoch 3/50
88130/88130 [==============================] - 43s 484us/step - loss: 1.6178 - acc: 0.5080
Epoch 4/50
88130/88130 [==============================] - 61s 694us/step - loss: 1.5742 - acc: 0.5187
Epoch 5/50
88130/88130 [==============================] - 55s 624us/step - loss: 1.5492 - acc: 0.5241
Epoch 6/50
88130/88130 [==============================] - 56s 635us/step - loss: 1.5205 - acc: 0.5332
Epoch 7/50
88130/88130 [==============================] - 61s 696us/step - loss: 1.5013 - acc: 0.5371
Epoch 8/50
88130/88130 [==============================] - 61s 690us/step - loss: 1.4870 - acc: 0.5431
Epoch 9/50
88130/88130 [==============================] - 46s 519us/step - loss: 1.4697 - acc: 0.5464
Epoch 10/50
88130/88130 [==============================] - 51s 574us/step - loss: 

In [ ]:
Epoch 1/50
88130/88130 [==============================] - 51s 573us/step - loss: 1.7970 - acc: 0.4644
Epoch 2/50
88130/88130 [==============================] - 33s 369us/step - loss: 1.6771 - acc: 0.4935
Epoch 3/50
88130/88130 [==============================] - 43s 484us/step - loss: 1.6178 - acc: 0.5080
Epoch 4/50
88130/88130 [==============================] - 61s 694us/step - loss: 1.5742 - acc: 0.5187
Epoch 5/50
88130/88130 [==============================] - 55s 624us/step - loss: 1.5492 - acc: 0.5241
Epoch 6/50
88130/88130 [==============================] - 56s 635us/step - loss: 1.5205 - acc: 0.5332
Epoch 7/50
88130/88130 [==============================] - 61s 696us/step - loss: 1.5013 - acc: 0.5371
Epoch 8/50
88130/88130 [==============================] - 61s 690us/step - loss: 1.4870 - acc: 0.5431
Epoch 9/50
88130/88130 [==============================] - 46s 519us/step - loss: 1.4697 - acc: 0.5464
Epoch 10/50
88130/88130 [==============================] - 51s 574us/step - loss: 1.4608 - acc: 0.54881s - 
Epoch 11/50
88130/88130 [==============================] - 46s 527us/step - loss: 1.4532 - acc: 0.5496
Epoch 12/50
88130/88130 [==============================] - 48s 544us/step - loss: 1.4473 - acc: 0.5517
Epoch 13/50
88130/88130 [==============================] - 51s 583us/step - loss: 1.4431 - acc: 0.5526
Epoch 14/50
88130/88130 [==============================] - 54s 615us/step - loss: 1.4380 - acc: 0.5540
Epoch 15/50
88130/88130 [==============================] - 59s 672us/step - loss: 1.4388 - acc: 0.5534
Epoch 16/50
88130/88130 [==============================] - 60s 680us/step - loss: 1.4402 - acc: 0.5535
Epoch 17/50
88130/88130 [==============================] - 64s 730us/step - loss: 1.4389 - acc: 0.55206s - loss: 1.4326 - acc - ETA: 5s - ETA: 3s - l - ETA: 0s - loss: 1.4381 - a
Epoch 18/50
88130/88130 [==============================] - 69s 779us/step - loss: 1.4424 - acc: 0.5545
Epoch 19/50
88130/88130 [==============================] - 72s 821us/step - loss: 1.4461 - acc: 0.5529
Epoch 20/50
88130/88130 [==============================] - 75s 853us/step - loss: 1.4475 - acc: 0.5515
Epoch 21/50
88130/88130 [==============================] - 78s 886us/step - loss: 1.4551 - acc: 0.5500
Epoch 22/50
88130/88130 [==============================] - 80s 910us/step - loss: 1.4483 - acc: 0.5510
Epoch 23/50
88130/88130 [==============================] - 83s 938us/step - loss: 1.4501 - acc: 0.5537
Epoch 24/50
88130/88130 [==============================] - 86s 971us/step - loss: 1.4599 - acc: 0.5492
Epoch 25/50
88130/88130 [==============================] - 89s 1ms/step - loss: 1.4617 - acc: 0.5495
Epoch 26/50
88130/88130 [==============================] - 92s 1ms/step - loss: 1.4742 - acc: 0.5475
Epoch 27/50
88130/88130 [==============================] - 95s 1ms/step - loss: 1.4748 - acc: 0.5486
Epoch 28/50
88130/88130 [==============================] - 99s 1ms/step - loss: 1.4760 - acc: 0.5477
Epoch 29/50
88130/88130 [==============================] - 101s 1ms/step - loss: 1.4866 - acc: 0.5422
Epoch 30/50
88130/88130 [==============================] - 107s 1ms/step - loss: 1.4939 - acc: 0.5429
Epoch 31/50
88130/88130 [==============================] - 107s 1ms/step - loss: 1.5019 - acc: 0.5418
Epoch 32/50
88130/88130 [==============================] - 111s 1ms/step - loss: 1.5027 - acc: 0.5407
Epoch 33/50
88130/88130 [==============================] - 115s 1ms/step - loss: 1.5082 - acc: 0.5392
Epoch 34/50
88130/88130 [==============================] - 118s 1ms/step - loss: 1.5045 - acc: 0.5412
Epoch 35/50
88130/88130 [==============================] - 123s 1ms/step - loss: 1.5203 - acc: 0.5377 0s - loss: 1.5202 - acc: 0.537
Epoch 36/50
88130/88130 [==============================] - 128s 1ms/step - loss: 1.5202 - acc: 0.5355
Epoch 37/50
88130/88130 [==============================] - 131s 1ms/step - loss: 1.5253 - acc: 0.5362 12s - los - ETA: 2s - loss: 1.5238 
Epoch 38/50
88130/88130 [==============================] - 133s 2ms/step - loss: 1.5366 - acc: 0.5331A: 2:04 - loss: - ET - ETA: 0s - loss: 1.5361 - acc: 0
Epoch 39/50
88130/88130 [==============================] - 136s 2ms/step - loss: 1.5311 - acc: 0.5348
Epoch 40/50
88130/88130 [==============================] - 139s 2ms/step - loss: 1.5399 - acc: 0.5338
Epoch 41/50
88130/88130 [==============================] - 144s 2ms/step - loss: 1.5524 - acc: 0.5317
Epoch 42/50
88130/88130 [==============================] - 146s 2ms/step - loss: 1.5625 - acc: 0.5285
Epoch 43/50
88130/88130 [==============================] - 149s 2ms/step - loss: 1.5628 - acc: 0.5263
Epoch 44/50
88130/88130 [==============================] - 153s 2ms/step - loss: 1.5592 - acc: 0.5289
Epoch 45/50
88130/88130 [==============================] - 156s 2ms/step - loss: 1.5581 - acc: 0.5290
Epoch 46/50
88130/88130 [==============================] - 161s 2ms/step - loss: 1.5783 - acc: 0.5234
Epoch 47/50
88130/88130 [==============================] - 165s 2ms/step - loss: 1.5790 - acc: 0.5235
Epoch 48/50
88130/88130 [==============================] - 167s 2ms/step - loss: 1.5707 - acc: 0.5254
Epoch 49/50
88130/88130 [==============================] - 169s 2ms/step - loss: 1.5808 - acc: 0.5243
Epoch 50/50
88130/88130 [==============================] - 173s 2ms/step - loss: 1.5876 - acc: 0.5220

----- temperature parameter: 1.5
----- Generating with seed: "shall i compare thee to a summer's day?
"
shall i compare thee to a summer's day?
disired doth loten night afterwired?
  loot is a full eyed, ors apvinved gaare.
when thrust altiming eare dwelll's opers am ofunliwo,
be mistage weere's steetlciore flowers comose,
or aletk knows thee brasted heoflery unoth.
o thanrany, it mogchs sways by tonel.
and give anture motmeint hims of fers placrine's,
alonging not tack goes threwn hath far time)
not rered jewel prisont leave to avermf aummer
edtr'i hrow nane sor five's ead,
using gained anule conben noom she curi,
numing time sliaves in stees whit inwaliarly,
deov'nsetfcl my e'ey eyide with in thee,


----- temperature parameter: 0.75
----- Generating with seed: "shall i compare thee to a summer's day?
"
shall i compare thee to a summer's day?
  wild far to steal at be no blering beaut.
  but to me a woettuen dwill, and so founts
sumaect the might and again to less in that is shade,
the that lawcing with on some deade mine,
but shall to be are even my gone,
and by yourse is is sad pity,
my heart truths shadow my fair faili aj in do brojenf for besiker,
a joutiin thy hath in his his mad, staig,
being the would worths stop at not my self,
  and starign too all should bart the bean,
and therefore gracinaeck of end, own with eecw part.
to praise your do and a lome's and gracit,
and then age' de lies in the for thee but row,


----- temperature parameter: 0.25
----- Generating with seed: "shall i compare thee to a summer's day?
"
shall i compare thee to a summer's day?
and this not the seemed and the beauty of thee,
and thou my self this confeered to me,
to me a loss thou my all to parts and thought,
and this in thee art my thee to the seemed:
then i shame hath with the seeming thy seee wind,
and thou my self shall to parts, and and of thee,
and and my self thou art stand to my self,
and thou my self away, and stand to,
that i sea it me thee to the worth to be than my love,
  then my love and thou my love and thee,
and thou my self though the seemen to steal to cannot to thee,
that i am shall in thee to see are to with a come,
and and thou my love and that to show,